Load your pretrained model

In [ ]:
from transformers import BertTokenizer, BertForMaskedLM
#from torch.nn import functional as F
#import torch

# #a szavak tokenizálója
tokenizer = BertTokenizer.from_pretrained("SZTAKI-HLT/hubert-base-cc")
#model = BertForMaskedLM.from_pretrained("SZTAKI-HLT/hubert-base-cc") #comment out if you are yousing your model!!

In [ ]:
PATH = "betanitas 8 25 epoch 64 batch mondat"

from transformers import pipeline

fill_mask = pipeline(
    "fill-mask",
    #model="/content/drive/MyDrive/12 epoch model/",
    model=BertForMaskedLM.from_pretrained(PATH,local_files_only=True), #load your model!!!
    tokenizer=BertTokenizer.from_pretrained("SZTAKI-HLT/hubert-base-cc",local_files_only=False)
)

**The algorithm** - random master

Iterate on a sentence masking one word at a time and predicting one of the 5 most likely words, then continue the iteration with that sentence (one word potentially swapped).

In [ ]:
# a generálómappa elérési útja
genetaringfolderpath='./' #figyelem per jel!!
#genetaringfolderpath='/content/drive/MyDrive/PyNLP/'

In [ ]:
#a bemeneti szöveg betöltése
with open(genetaringfolderpath+'bemeneti_es_randomizalt_szoveg.txt') as file_in:
    text = file_in.read()

In [ ]:
#a csuszoablak szamlalaojanak betoltese
with open(genetaringfolderpath+'csuszoablak_counter.txt') as file_in_k:
    k = file_in_k.read()
k=int(k)
print(k)

In [ ]:
text

Parameters

In [ ]:
random_pred_parameter=20
#goal_masked_words=len(text.split())/2 #random cserélgetünk, átlagban minden 2. szót

# tokens or words we don't want in our generated sentences
excluded_words = ['[ P A D ]', '[ U N K ]', ';', '.', ':', '?', '!', ',', '"', '#', '$', '[', ']', '%', '&', '(', ')', '*', '+', '-', '/', '0',  '1', '2', '3', '4', '5', '6', '7', '8', '9', '<', '=', '>', '<', '”', '«', '»', '–', '“', '„', ] 

In [ ]:
import random

In [ ]:
def modify_text(sentence, exc_words, n_iter, prob=0, random_pred_parameter=20):
  #sentence: the text to modify, generate from
  #exc_words: words, tokens we want to avoid when generating the new texts
  #n_iter: how many times we want to swap tokens
  #prob: in each iteration we add a mask token at the end of the sentence with this probability
  #all_sentences = [sentence] #saving all the generated sentences in the process

  while n_iter > 0:

    sentence_words = sentence.split()
    sentence_words[random.choice(range(len(sentence_words)))] = tokenizer.mask_token
    masked_sentence = " ".join(sentence_words)
    chosen_sentence = fill_mask(masked_sentence, top_k=random_pred_parameter)[random.choice(range(random_pred_parameter))]

    while chosen_sentence['token_str'] in excluded_words:
      chosen_sentence = fill_mask(masked_sentence, top_k=random_pred_parameter)[random.choice(range(random_pred_parameter))]
    sentence = chosen_sentence['sequence']
    k = random.random()
    if k < prob:
      sentence = sentence + ' ' + tokenizer.mask_token
      chosen_sentence = fill_mask(sentence, top_k=random_pred_parameter)[random.choice(range(random_pred_parameter))]

      while bool([exclword for exclword in excluded_words if(exclword in a['token_str'])]):
        chosen_sentence = fill_mask(sentence, top_k=random_pred_parameter)[random.choice(range(random_pred_parameter))]
      sentence = chosen_sentence['sequence']
    #all_sentences.append(sentence) # not yet returned by the function
    n_iter -= 1
  return str(sentence)

Itt 64 szavasra vettem a szövegszakaszokat, mert a meglévő szöveg kevesebb mint 256 szó. De ez a szám átállítható 512-re

In [ ]:
n_iter=(128/2) #azaz az ablakon belül átag minden 2. szót letakarjuk random módon persze

In [ ]:
input_len = len(text.split())
while k<input_len/128:
    text_2 = text.split()[k*128:(k+1)*128] #egyszerre mindig csak egy 64 szavas részleten futtatjuk a függvényt, majd ezt hozzáírjuk a kimenethez
    text_2=" ".join(text_2)
    generated = modify_text(text_2, excluded_words, n_iter,random_pred_parameter=20)
    with open(genetaringfolderpath+'randomizalt_kimenet.txt', 'a') as file_out:
        file_out.write(generated)
        file_out.write('.')
    k+=1
    with open(genetaringfolderpath+'csuszoablak_counter.txt', 'w') as file_out_k:
        file_out_k.write(str(k))
    print("progress :")
    print(k/(input_len/128))

In [ ]:
input_len